In [1]:

import requests
import json

#read the config json file
with open('salesforce_config.json', 'r') as f:
    C = json.load(f)

# Set the Salesforce URL,client ID, client secret, username and password from configfile   

salesforce_url = "https://login.salesforce.com/services/oauth2/token"
client_id = C["CLIENTID"]
client_secret = C["CLIENTSECRET"]
username = C["USERNAME"]
password = C["PASSWORD"]


# Set the REST API endpoint URL for obtaining an access token
rest_api_url = salesforce_url+"/services/oauth2/token"

payload = {
    "grant_type":"password",
    "client_id": client_id,
    "client_secret":client_secret,
    "username": username,
    "password": password
}

# Send the POST request to the REST API endpoint with the payload
response = requests.post(salesforce_url, data=payload)
print('Done')
print(response.status_code)
print(response.reason)

response.json()




Done
200
OK


{'access_token': '00DDn000002q7Dy!ARkAQO7D9HV9l.CAvUmzM2yGUXlaEf3p5kpzFlM5TwWj.XCaXZXm3VoMSspnlTBSQRul37xogO56XJO.LQb2OFkKn3Ys9wXi',
 'instance_url': 'https://koresoftware-dev-ed.develop.my.salesforce.com',
 'id': 'https://login.salesforce.com/id/00DDn000002q7DyMAI/005Dn000002M1PFIA0',
 'token_type': 'Bearer',
 'issued_at': '1678464094300',
 'signature': '6zr3PthNg3Jz8xups+qkysguHByoto8UXBjv84Q6nb4='}

In [2]:
access_token = response.json().get("access_token")
instance_url = response.json().get("instance_url")

# Set the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Define the SOQL query to retrieve data from Contacts Table in Salesforce
#query = """Select IndividualId,OwnerId,Email,Suffix,FirstName,LastName,MailingStreet,MailingCity,MailingState,
 #          MailingPostalCode,MailingCountry,DoNotCall,Phone,MobilePhone from Contact where DoNotCall<>True"""

query = """Select IndividualId,OwnerId,Email,FirstName,LastName,MailingStreet,MailingCity,MailingState,
            MailingPostalCode,MailingCountry,DoNotCall,Phone,MobilePhone from Contact where DoNotCall<>True """

# Construct the URL
url = f"{instance_url}/services/data/v51.0/query?q={query}"

# Make the request
response = requests.get(url, headers=headers)

# Process the response
if response.status_code == 200:
    data = response.json()
    with open("output_notTrue.json", "w") as f:
        json.dump(data, f)
else:
    print(f"Error retrieving data: {response.status_code}")
    


In [3]:
# Validating the total number of records in the source
query = "select count() from Contact"
url = f"{instance_url}/services/data/v51.0/query?q={query}"
# Make the request
response = requests.get(url, headers=headers)
response.json()

{'totalSize': 1495, 'done': True, 'records': []}

In [4]:
# Validating the total number of records in the source with donot call = "true"
query = "select count() from Contact where DoNotCall = True"
url = f"{instance_url}/services/data/v51.0/query?q={query}"
# Make the request
response = requests.get(url, headers=headers)
response.json()

{'totalSize': 8, 'done': True, 'records': []}